In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle competitions download -c titanic

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /content. Or use the environment method.


## Read Data

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train.head()

In [ ]:
test.head()

## Visualize Data

In [ ]:
#sns.countplot(x='Survived',data=train)
#sns.countplot(x='Survived',hue='Sex',data=train)
#sns.countplot(x='Survived',hue="Pclass",data=train)
#sns.distplot(train['Age'].dropna(),kde=False,bins=30)
#train.info()
#sns.countplot(x='SibSp',data=train)
#train['Fare'].hist(bins=40,figsize=(10,4))

## Feature Engineering

In [ ]:
dfs = [train, test]
#sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')
#sns.heatmap(test.isnull(),yticklabels=False,cbar=False,cmap='viridis')

Fare

In [ ]:
test[pd.isnull(test['Fare'])]
fare_fill = test[test['Pclass']==3]['Fare'].mean() # Mean of Fare values when Pclass = 3
test['Fare'].fillna(fare_fill, inplace=True)

Age

In [ ]:
#sns.boxplot(x='Pclass',y='Age',data=train)
for df in dfs:
    def impute_age(col): # Use Pclass to fill null age values
        age = col[0]
        pclass = col[1]

        if (pd.isnull(age)):
            return df[df['Pclass']==pclass]['Age'].median()
        return age

    df['Age'] = df[['Age', 'Pclass']].apply(impute_age, axis=1)

Encoding (Sex, Embarked, Cabin)

Note: this can't be done using a for loop since we're changing the variable reference

In [ ]:
for df in dfs:
    df['Cabin_letter'] = df['Cabin'].dropna().apply(lambda x : x[0])
#train['Cabin_letter'].value_counts()
#test['Cabin_letter'].value_counts()
sex = pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True, prefix='Emb')
cabin = pd.get_dummies(train['Cabin_letter'],prefix='Cabin')
train = pd.concat([train,sex,embark,cabin],axis=1)
sex = pd.get_dummies(test['Sex'],drop_first=True)
embark = pd.get_dummies(test['Embarked'],drop_first=True, prefix='Emb')
cabin = pd.get_dummies(test['Cabin_letter'],prefix='Cabin')
test = pd.concat([test,sex,embark,cabin],axis=1)
test['Cabin_T'] = 0 # Makes sure train and test have same # of columns

Name

Note: also can't be done using a for loop bc adding a column doesn't update in the original reference for some reason

In [ ]:
def get_title(x):
    for i in x.split():
        if i[-1] == '.':
            return i[:-1]

#train['Name'].apply(get_title).value_counts()
#test['Name'].apply(get_title).value_counts()
title_map = {
    'Mr': 0, 'Miss': 0, 'Mrs': 0, 'Ms': 0, 'Mme': 0, 'Mlle': 0,
    'Lady': 1, 'Sir': 1, 'Dr': 1, 'Rev': 1, 'Don': 1, 'Dona': 1, 'Countess': 1,
    'Jonkheer': 1, 'Capt': 1, 'Col': 1, 'Major': 1, 'Master': 1,
}

train['Rank'] = train['Name'].apply(get_title).map(title_map)
test['Rank'] = test['Name'].apply(get_title).map(title_map)

Remove extra columns

In [ ]:
#train.columns
train.drop(['PassengerId', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Cabin_letter'], axis=1, inplace=True)
#train.head()
test.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Cabin_letter'], axis=1, inplace=True)
#test.head()
#sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')
#sns.heatmap(test.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
train.columns[1:]

## KNN Model

### Standardizing Variables

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(train.drop('Survived',axis=1))

In [ ]:
scaled_features = scaler.transform(train.drop('Survived',axis=1))

In [ ]:
df_feat = pd.DataFrame(scaled_features,columns=train.columns[1:])
df_feat.head()

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features, train['Survived'], test_size=0.3)

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
pred = knn.predict(X_test)

### Evaluate KNN Model

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

### Choosing K

In [ ]:
error_rate = []

for i in range(1,40):

    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('WITH K=1')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=8) # Change k as needed

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('WITH K=8')  # Label k as needed
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

## Submit Model Using Test Data

In [ ]:
test.head()

In [ ]:
final_X = test.drop('PassengerId', axis=1)
final_pred = knn.predict(final_X)

In [ ]:
final = pd.DataFrame()
final['PassengerId'] = test['PassengerId']
final['Survived'] = final_pred
final.to_csv('pred.csv', index=False)